In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nd = pd.read_csv('numeric_data.csv')


output = ["stais1","stais3","stais4","stais5","stais6","stais7","stais8","stais9","stais10","stais11","stais12","stais13",
          "stais14","stais15","stais16","stais17","stais18","stais19","stais20",
          "sds_1","sds_3","sds_4","stait5","sds_6","sds_7","sds_8","sds_9","sds_10","sds_11","sds_12","sds_13",
          "sds_14","sds_15","sds_16","sds_17","sds_18","sds_19","sds_20"]

data_y = nd.loc[:,output]
data_x = nd

columns = nd.columns.values

for col in columns:
    if col in output:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif nd[col].isna().sum() != 0:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif nd[col].dtypes == object:
        data_x = data_x.drop([col],axis = 1)
        #print(nd[col].dtypes)

    '''
    elif not nd[col].dtypes is np.dtype('int64'):
        if not nd[col].dtypes is np.dtype('float64'):
            data_x = data_x.drop([col],axis = 1)
            print(nd[col].dtypes)
    ''' 

data_y = nd.loc[:,["stai_s_score"]]

data_x = np.array(data_x, dtype=np.float32)
data_y = np.array(data_y, dtype=np.float32)


X = data_x[:1316]
y = data_y[:1316]
X_test = data_x[1316:]
y_test = data_y[1316:]

print(data_x.shape)
print(data_y.shape)
#print(data_t.shape)

print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)


(1463, 287)
(1463, 1)
(1316, 287)
(1316, 1)
(147, 287)
(147, 1)


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


x1 = np.delete(X,slice(263),0)
x2 = np.delete(X,slice(263,526),0)
x3 = np.delete(X,slice(526,790),0)
x4 = np.delete(X,slice(790,1053),0)
x5 = X[:1053]


y1 = np.delete(y,slice(263),0)
y2 = np.delete(y,slice(263,526),0)
y3 = np.delete(y,slice(526,790),0)
y4 = np.delete(y,slice(790,1053),0)
y5 = y[:1053]

X_groups = [x1,x2,x3,x4,x5]
y_groups = [y1,y2,y3,y4,y5]
X_vali = [X[:263],X[263:526],X[526:790],X[790:1053],X[1053:]]
y_vali = [y[:263],y[263:526],y[526:790],y[790:1053],y[1053:]]


class TwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


input_size = X.shape[1]
hidden_size = 100
output_size = y.shape[1]

learning_rate = 0.01
num_epochs = 100


# Training
mse = 0
for i in range(5):
    X_tensor = torch.tensor(X_groups[i])
    y_tensor = torch.tensor(y_groups[i])
    
    X_validation = torch.tensor(X_vali[i])
    y_validation = torch.tensor(y_vali[i])
    
    
    model = TwoLayerNN(input_size, hidden_size, output_size)

    
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    

    for epoch in range(num_epochs):
        
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
    
    predictions = model(X_validation)
    loss = criterion(predictions, y_validation)
    print("Validation loss:",str(loss.item()))
    mse += loss.item()

CV_k = mse / 5
print(CV_k)

Epoch 0, Loss: 1551.88818359375
Epoch 20, Loss: 127.3543472290039
Epoch 40, Loss: 81.0678939819336
Epoch 60, Loss: 50.23618698120117
Epoch 80, Loss: 28.785600662231445
Validation loss: 23.877710342407227
Epoch 0, Loss: 1905.4002685546875
Epoch 20, Loss: 92.2089614868164
Epoch 40, Loss: 47.75111389160156
Epoch 60, Loss: 27.342304229736328
Epoch 80, Loss: 73.41136169433594
Validation loss: 20.455781936645508
Epoch 0, Loss: 2219.544677734375
Epoch 20, Loss: 91.51712036132812
Epoch 40, Loss: 43.068782806396484
Epoch 60, Loss: 22.25960350036621
Epoch 80, Loss: 18.97026824951172
Validation loss: 7.805393218994141
Epoch 0, Loss: 2510.192626953125
Epoch 20, Loss: 263.2973937988281
Epoch 40, Loss: 121.76488494873047
Epoch 60, Loss: 83.77448272705078
Epoch 80, Loss: 51.15550994873047
Validation loss: 29.800073623657227
Epoch 0, Loss: 1998.2796630859375
Epoch 20, Loss: 59.941707611083984
Epoch 40, Loss: 31.050981521606445
Epoch 60, Loss: 17.67165756225586
Epoch 80, Loss: 11.452853202819824
Valida

In [14]:
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)


model = TwoLayerNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)


with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)

#print(predictions)
print(f"Test Loss: {test_loss.item()}")


Epoch 0, Loss: 1859.7412109375
Epoch 20, Loss: 96.57730865478516
Epoch 40, Loss: 55.55731201171875
Epoch 60, Loss: 33.16300582885742
Epoch 80, Loss: 17.905309677124023
Test Loss: 10.362555503845215


In [17]:
def scores(data):
    return [sum(((5-data[0]),(data[1]),(data[2]),(5-data[3]),(data[4]),(data[5]),(5-data[6]),(data[7]),
                (5-data[8]),(5-data[9]),(data[10]),(data[11]),(data[12]),(5-data[13]),(5-data[14]),
                (data[15]),(data[16]),(5-data[17]),(5-data[18]))),
                sum(((data[19]),(data[20]),(data[21]),(5-data[22]),(5-data[23]),(data[24]),(data[25]),(data[26]),
                (data[27]),(5-data[28]),(5-data[29]),(data[30]),(5-data[31]),(data[32]),(5-data[33]),
                (5-data[34]),(5-data[35]),(data[36]),(5-data[37])))
            ]


#print([int(i+0.5) for i in predictions[0]])
#print(y_test[0].shape)

predict = predictions.tolist()
print(predict)


p_scores = [scores(i) for i in predict]




y_scores = [scores(i) for i in y_test]
#print(p_scores)

p_sc = [[int(i[0]+0.5),int(i[1]+0.5)] for i in p_scores]
y_sc = [[int(j[0]),int(j[1])] for j in y_scores]
print(p_sc)
print()
print(y_sc)


p_s = torch.tensor(p_scores)
y_s = torch.tensor(y_scores)


l = criterion(p_s, y_s)
print("\nMSEloss:")
print(l.item())

[[44.771602630615234], [37.5458984375], [50.87834167480469], [26.459321975708008], [49.37443161010742], [43.851566314697266], [61.485572814941406], [16.25452423095703], [24.365066528320312], [33.29551315307617], [51.09169387817383], [19.973522186279297], [41.0020866394043], [27.689584732055664], [33.83716583251953], [49.79103469848633], [54.041568756103516], [29.903751373291016], [51.23584747314453], [49.96720504760742], [36.328125], [67.73301696777344], [37.542320251464844], [48.14918899536133], [29.215150833129883], [34.305992126464844], [57.37241744995117], [41.89249801635742], [29.73622703552246], [34.895530700683594], [48.07067108154297], [60.03105545043945], [50.140846252441406], [37.958709716796875], [33.681983947753906], [27.27735710144043], [51.0291862487793], [40.351844787597656], [43.5291748046875], [38.349857330322266], [36.88145065307617], [41.106937408447266], [24.444599151611328], [43.16530227661133], [43.36847686767578], [39.7169189453125], [20.493534088134766], [42.578

IndexError: list index out of range